In [1]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np

from chemistry import Atom, Bond, Molecule

In [2]:
structures = pd.read_feather('../data/structures.feather')
labelled = pd.read_feather('../data/train.feather')
unlabelled = pd.read_feather('../data/test.feather')

/home/gautham/apps/anaconda3/envs/kaggle/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [3]:
def merge(data):
    data.copy()
    
    m0 = data.merge(structures, left_on=['molecule_name', 'atom_index_0'], right_on=['molecule_name', 'atom_index'], suffixes=('0', '0'))
    m1 = data.merge(structures, left_on=['molecule_name', 'atom_index_1'], right_on=['molecule_name', 'atom_index'], suffixes=('1', '1'))
    
    l0 = m0[['x', 'y', 'z']]
    l1 = m1[['x', 'y', 'z']]
    d = l0 - l1
    d2 = d * d
    dist2 = d2.x + d2.y + d2.z
    dist = dist2.apply(np.sqrt)
    dist.name = 'distance'
    
    merged = data.join(dist)
    merged['atom_0'] = m0.atom
    merged['atom_1'] = m1.atom
    
    return merged

In [4]:
labelled_merged = merge(labelled)
unlabelled_merged = merge(unlabelled)

In [5]:
min_dist = min(labelled_merged.distance.min(), unlabelled_merged.distance.min())
max_dist = max(labelled_merged.distance.max(), unlabelled_merged.distance.max())

min_coeff = labelled_merged.scalar_coupling_constant.min()
max_coeff = labelled_merged.scalar_coupling_constant.max()

In [6]:
offset = min_coeff
scale = max_coeff - min_coeff

labelled_merged['norm_distance'] = (labelled_merged.distance - min_dist) / (max_dist - min_dist)
unlabelled_merged['norm_distance'] = (unlabelled_merged.distance - min_dist) / (max_dist - min_dist)

labelled_merged['norm_scc'] = (labelled_merged.scalar_coupling_constant - min_coeff) / (max_coeff - min_coeff)

In [7]:
types = labelled.type.unique()
atoms = structures.atom.unique()

index = 0
type_index = {}
for t in types:
    type_index[t] = index
    index += 1

atom_index = {}
for a in atoms:
    atom_index[a] = index
    index += 2

dist_index = index
index += 1

columns = index

In [8]:
def partition_labelled(data, count=None, train_frac=0.7):
    n_labelled = count if count is not None else len(labelled)
    n_train = int(n_labelled * train_frac)
    n_test = n_labelled - n_train
    indices = np.arange(0, n_labelled)
    np.random.shuffle(indices)
    
    train_indices = indices[0:n_train]
    test_indices = indices[n_train:]
    
    train = data.iloc[train_indices, :]
    test = data.iloc[test_indices, :]

    return train, test

def make_input(data, columns, type_index, atom_index, dist_index):
    n = len(data)
    input = np.zeros((columns, n), dtype='float32')
    
    for t in type_index:
        input[type_index[t], data.type == t] = 1
        
    for a in atom_index:
        input[atom_index[a], data.atom_0 == t] = 1
        input[atom_index[a] + 1, data.atom_1 == t] = 1
        
    input[dist_index] = data.norm_distance
        
    return input.T

def make_output(data):
    n = len(data)
    output = np.zeros(n, dtype='float32')
    output[:] = data.norm_scc
        
    return output

In [9]:
from sklearn.svm import SVR,LinearSVR

In [10]:
labelled_merged.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'type',
       'scalar_coupling_constant', 'distance', 'atom_0', 'atom_1',
       'norm_distance', 'norm_scc'],
      dtype='object')

In [15]:
labelled_train, labelled_test = partition_labelled(labelled_merged, 10000)

labelled_train_input = make_input(labelled_train, columns, type_index, atom_index, dist_index)
labelled_train_output = make_output(labelled_train)

labelled_test_input = make_input(labelled_test, columns, type_index, atom_index, dist_index)
labelled_test_output = make_output(labelled_test)

labelled_train_input.shape, labelled_train_output.shape

((7000, 19), (7000,))

In [16]:
model = LinearSVR()
model.fit(labelled_train_input, labelled_train_output)
print('')
#print(model.predict(test_input))
#print(test_output)
model.score(labelled_test_input, labelled_test_output)

/home/gautham/apps/anaconda3/envs/kaggle/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.915424362556078

In [20]:
labelled_train_input = make_input(labelled_merged, columns, type_index, atom_index, dist_index)
labelled_train_output = make_output(labelled_merged)

labelled_train_input.shape, labelled_train_output.shape

((4658147, 19), (4658147,))

In [21]:
model = LinearSVR()
model.fit(labelled_train_input, labelled_train_output)

/home/gautham/apps/anaconda3/envs/kaggle/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
          random_state=None, tol=0.0001, verbose=0)

In [22]:
unlabelled_input = make_input(unlabelled_merged, columns, type_index, atom_index, dist_index)

In [23]:
unlabelled_output = model.predict(unlabelled_input)

In [24]:
unlabelled_output

array([0.14489314, 0.51499623, 0.16347423, ..., 0.15980076, 0.14425363,
       0.51435673])

In [25]:
unlabelled_output = unlabelled_output * scale + offset

In [26]:
unlabelled_output

array([-1.28506886, 87.9462694 ,  3.19480777, ...,  2.30913835,
       -1.4392517 , 87.79208655])

In [27]:
output_df = pd.DataFrame({'id':unlabelled.id, 'scalar_coupling_constant':pd.Series(unlabelled_output, index=unlabelled.index)})

In [28]:
output_df

id  scalar_coupling_constant
0        4658147                 -1.285069
1        4658148                 87.946269
2        4658149                  3.194808
3        4658150                 87.946269
4        4658151                 -1.439252
5        4658152                 87.867251
6        4658153                  2.384303
7        4658154                -11.825081
8        4658155                -11.825081
9        4658156                 87.867251
10       4658157                  2.384303
11       4658158                -11.720238
12       4658159                 88.014551
13       4658160                  2.531603
14       4658161                  2.384303
15       4658162                 87.867251
16       4658163                -11.825081
17       4658164                -11.732379
18       4658165                  2.556298
19       4658166                 88.039247
20       4658167                -11.900245
21       4658168                  2.309138
22       4658169                 87.792087
23       4658170                 87.866033
24       4658171                 -1.365305
25       4658172                 -1.365305
26       4658173                -11.826298
27       4658174                  3.196333
28       4658175                  3.196333
29       4658176                  3.274670
...          ...                       ...
2505512  7163659                 88.020096
2505513  7163660                 -1.211242
2505514  7163661                  2.535078
2505515  7163662                  2.535078
2505516  7163663                 -1.213312
2505517  7163664                  3.348326
2505518  7163665                  3.348326
2505519  7163666                  2.539540
2505520  7163667                 -1.208850
2505521  7163668                 88.152284
2505522  7163669                  2.467642
2505523  7163670                  2.467642
2505524  7163671                 -1.280748
2505525  7163672                  3.280890
2505526  7163673                  2.538739
2505527  7163674                  1.375015
2505528  7163675                  2.526845
2505529  7163676                  2.526845
2505530  7163677                  2.526845
2505531  7163678                 -1.221545
2505532  7163679                 87.946492
2505533  7163680                 -1.284846
2505534  7163681                  3.276792
2505535  7163682                  0.200446
2505536  7163683                  2.382813
2505537  7163684                 -1.252606
2505538  7163685                 -1.252398
2505539  7163686                  2.309138
2505540  7163687                 -1.439252
2505541  7163688                 87.792087

[2505542 rows x 2 columns]

In [29]:
output_df.to_csv('../data/pred.csv',index=False)